# Use O2C_Template Customized Notebook Template

In [1]:
import pandas as pd
import numpy as np

In [2]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [3]:
table_name = 'ORDER_TO_CASH_ENRICHED'

sf_df = my_session.sql("select * from {}".format(table_name))
df_final = sf_df.to_pandas()

In [4]:
df_train = df_final[df_final['INVOICESTATUS'] == 'Paid']
df_new = df_final[df_final['INVOICESTATUS'] != 'Paid']

In [5]:
df_train.shape, df_new.shape

((170290, 68), (3001, 68))

In [7]:
df = df_final.copy()

In [8]:
# Assuming df is your dataframe
df['ORDERDATE'] = pd.to_datetime(df['ORDERDATE'], errors='coerce')
df['DELIVERYDATE'] = pd.to_datetime(df['DELIVERYDATE'], errors='coerce')
df['INVOICEDATE'] = pd.to_datetime(df['INVOICEDATE'], errors='coerce')
df['PAYMENTDATE'] = pd.to_datetime(df['PAYMENTDATE'], errors='coerce')
df['INVOICEDUEDATE'] = pd.to_datetime(df['INVOICEDUEDATE'], errors='coerce')
df['DELIVEREDON'] = pd.to_datetime(df['DELIVEREDON'], errors='coerce')

In [9]:
df['MONTH'] = pd.to_datetime(df['ORDERDATE']).dt.month
df['YEAR'] = pd.to_datetime(df['ORDERDATE']).dt.year
df['WEEK'] = pd.to_datetime(df['ORDERDATE']).dt.isocalendar().week
df['WEEKEND'] = pd.to_datetime(df['ORDERDATE']).dt.weekday >= 5  # True if the day is Saturday or Sunday
df['DAY_OF_THE_MONTH'] = pd.to_datetime(df['ORDERDATE']).dt.day

In [10]:
# Delivery Delay
df['DELIVERY_DELAY'] = (df['SHIPMENTDATE'] - df['DELIVEREDON']).dt.days

# Payment Delay
df['PAYMENT_DELAY'] = (df['PAYMENTDATE'] - df['INVOICEDUEDATE']).dt.days

In [11]:
df.columns

Index(['ORDERID', 'ORDERITEMID', 'PRODUCTID', 'PRODUCTNAME', 'ORDERQUANTITY',
       'UNITPRICE', 'ORDERVALUE', 'CUSTOMERID', 'CUSTOMERNAME',
       'PRODUCTCATEGORY', 'ORDERDATE', 'ORDERSTATUS', 'ORDERAMOUNT',
       'ORDERCOUNT', 'SHIPMENTID', 'SHIPMENTDATE', 'DELIVERYDATE',
       'DELIVEREDON', 'CARRIER', 'SHIPMENTSTATUS', 'INVOICEID', 'INVOICEDATE',
       'INVOICEDUEDATE', 'PAYMENTDATE', 'INVOICESTATUS', 'PAYMENTMETHOD',
       'PRODUCTTYPE', 'QUANTITY', 'NAME', 'COMPANYTYPE', 'CONTACTDETAILS',
       'EMAILDETAILS', 'ADDRESSDETAILS', 'ADMINDETAILS', 'CREDITLIMIT',
       'CREDITRATING', 'CUSTOMERTYPE', 'CC_CUSTOMER_LIFETIME_VALUE',
       'CC_ORDER_FREQUENCY', 'CC_AVERAGE_ORDER_VALUE', 'CC_RECENCY',
       'CC_PREFERRED_PAYMENT_METHOD', 'CC_PREFERRED_PRODUCT_CATEGORY',
       'CC_PREFERRED_PRODUCT_TYPE', 'CC_AVG_ORDER_PROCESSING_TIME',
       'CC_AVG_INVOICE_PROCESSING_TIME', 'CC_AVG_DELIVERY_DELAY',
       'CC_AVG_PAYMENT_DELAY', 'CC_TOTAL_ORDERS', 'CC_TOTAL_DELAYS',
       'CC

In [13]:
cols = ['ORDERQUANTITY','UNITPRICE','ORDERVALUE'
,'CC_CUSTOMER_LIFETIME_VALUE','CC_ORDER_FREQUENCY', 'CC_AVERAGE_ORDER_VALUE', 'CC_RECENCY','CC_AVG_ORDER_PROCESSING_TIME'
,'CC_AVG_INVOICE_PROCESSING_TIME', 'CC_AVG_DELIVERY_DELAY','CC_AVG_PAYMENT_DELAY', 'CC_TOTAL_ORDERS', 'CC_TOTAL_DELAYS'
,'CC_ORDER_CONSISTENCY', 'CC_INVOICE_CONSISTENCY','CC_DELIVERY_CONSISTENCY', 'CC_PAYMENT_CONSISTENCY'
,'PR_TOTAL_SALES_VOLUME', 'PR_TOTAL_SALES_VALUE','PR_AVG_ORDER_QUANTITY', 'PR_AVG_UNIT_PRICE', 'PR_NUMBER_OF_ORDERS'
,'PR_AVG_DELIVERY_TIME', 'PR_AVG_INVOICE_TIME', 'CAT_TOTAL_SALES_VOLUME','CAT_TOTAL_SALES_VALUE', 'CAT_AVG_ORDER_QUANTITY', 'CAT_AVG_UNIT_PRICE'
,'CAT_NUMBER_OF_ORDERS', 'CAT_AVG_DELIVERY_TIME','CAT_AVG_INVOICE_TIME','DELIVERY_DELAY','PAYMENT_DELAY'
,'MONTH', 'YEAR', 'WEEK', 'WEEKEND', 'DAY_OF_THE_MONTH']

In [14]:
df = df[cols]

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 173291 entries, 0 to 173290
Data columns (total 38 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   ORDERQUANTITY                   173291 non-null  int16  
 1   UNITPRICE                       173291 non-null  int32  
 2   ORDERVALUE                      173291 non-null  int32  
 3   CC_CUSTOMER_LIFETIME_VALUE      173291 non-null  int32  
 4   CC_ORDER_FREQUENCY              173291 non-null  int16  
 5   CC_AVERAGE_ORDER_VALUE          173291 non-null  float64
 6   CC_RECENCY                      173291 non-null  int8   
 7   CC_AVG_ORDER_PROCESSING_TIME    173291 non-null  float64
 8   CC_AVG_INVOICE_PROCESSING_TIME  173291 non-null  float64
 9   CC_AVG_DELIVERY_DELAY           173291 non-null  float64
 10  CC_AVG_PAYMENT_DELAY            173291 non-null  float64
 11  CC_TOTAL_ORDERS                 173291 non-null  int16  
 12  CC_TOTAL_DELAYS 

In [16]:
df.tail()

,ORDERQUANTITY,UNITPRICE,ORDERVALUE,CC_CUSTOMER_LIFETIME_VALUE,CC_ORDER_FREQUENCY,CC_AVERAGE_ORDER_VALUE,CC_RECENCY,CC_AVG_ORDER_PROCESSING_TIME,CC_AVG_INVOICE_PROCESSING_TIME,CC_AVG_DELIVERY_DELAY,...,CAT_NUMBER_OF_ORDERS,CAT_AVG_DELIVERY_TIME,CAT_AVG_INVOICE_TIME,DELIVERY_DELAY,PAYMENT_DELAY,MONTH,YEAR,WEEK,WEEKEND,DAY_OF_THE_MONTH
173286,772,299,230828,65953955,558,61068.476852,26,6.929630,8.069444,-1.450926,...,14445,8.534271,7.971280,0.0,-5.0,7,2023,27,False,4
173287,790,9,7110,13756619,548,12426.936766,24,6.952123,8.113821,-1.523035,...,19990,8.499787,7.972449,-2.0,0.0,8,2024,34,True,24
173288,421,3,1263,13756619,548,12426.936766,24,6.952123,8.113821,-1.523035,...,19990,8.499787,7.972449,-2.0,0.0,8,2024,34,True,24
173289,450,2,900,13756619,548,12426.936766,24,6.952123,8.113821,-1.523035,...,19990,8.499787,7.972449,-2.0,0.0,8,2024,34,True,24
173290,916,19,17404,64484457,554,57575.408036,25,7.101786,8.203571,-1.626786,...,14445,8.534271,7.971280,0.0,8.0,1,2024,1,False,2


In [18]:
from sklearn.preprocessing import StandardScaler

# Standardize the predictor variables
scaler = StandardScaler()

X_scaled = scaler.fit_transform(df.drop(columns=['DELIVERY_DELAY', 'PAYMENT_DELAY']))

df_scaled = pd.DataFrame(X_scaled, columns=df.drop(columns=['DELIVERY_DELAY', 'PAYMENT_DELAY']).columns)
df_scaled['DELIVERY_DELAY'] = df['DELIVERY_DELAY']
df_scaled['PAYMENT_DELAY'] = df['PAYMENT_DELAY']

In [19]:
df_scaled.head()

,ORDERQUANTITY,UNITPRICE,ORDERVALUE,CC_CUSTOMER_LIFETIME_VALUE,CC_ORDER_FREQUENCY,CC_AVERAGE_ORDER_VALUE,CC_RECENCY,CC_AVG_ORDER_PROCESSING_TIME,CC_AVG_INVOICE_PROCESSING_TIME,CC_AVG_DELIVERY_DELAY,...,CAT_NUMBER_OF_ORDERS,CAT_AVG_DELIVERY_TIME,CAT_AVG_INVOICE_TIME,MONTH,YEAR,WEEK,WEEKEND,DAY_OF_THE_MONTH,DELIVERY_DELAY,PAYMENT_DELAY
0,0.768522,-0.249351,-0.245039,-0.381590,1.139589,-0.387349,-0.662097,-1.432849,0.871990,-0.685103,...,1.161755,-0.013016,-0.316488,1.309940,-1.590644,1.243291,-0.632181,-0.536187,-3.0,0.0
1,0.622408,-0.249991,-0.273455,-0.381590,1.139589,-0.387349,-0.662097,-1.432849,0.871990,-0.685103,...,1.161755,-0.013016,-0.316488,1.309940,-1.590644,1.243291,-0.632181,-0.536187,-3.0,0.0
2,-0.546511,-0.250310,-0.300408,-0.381590,1.139589,-0.387349,-0.662097,-1.432849,0.871990,-0.685103,...,1.161755,-0.013016,-0.316488,1.309940,-1.590644,1.243291,-0.632181,-0.536187,-3.0,0.0
3,2.930659,-0.250470,-0.269707,-0.146407,1.506251,-0.165817,0.797586,-0.091813,1.026887,0.813034,...,-1.013455,0.146568,1.419836,-1.069112,1.207321,-1.221954,1.581826,-1.442385,-2.0,-5.0
4,1.439322,-0.249671,-0.236610,-0.146407,1.506251,-0.165817,0.797586,-0.091813,1.026887,0.813034,...,-1.013455,0.146568,1.419836,-1.069112,1.207321,-1.221954,1.581826,-1.442385,-2.0,-5.0


In [20]:
df_scaled.describe()

,ORDERQUANTITY,UNITPRICE,ORDERVALUE,CC_CUSTOMER_LIFETIME_VALUE,CC_ORDER_FREQUENCY,CC_AVERAGE_ORDER_VALUE,CC_RECENCY,CC_AVG_ORDER_PROCESSING_TIME,CC_AVG_INVOICE_PROCESSING_TIME,CC_AVG_DELIVERY_DELAY,...,CAT_NUMBER_OF_ORDERS,CAT_AVG_DELIVERY_TIME,CAT_AVG_INVOICE_TIME,MONTH,YEAR,WEEK,WEEKEND,DAY_OF_THE_MONTH,DELIVERY_DELAY,PAYMENT_DELAY
count,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,...,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,1.732910e+05,171043.000000,171043.000000
mean,-5.742451e-17,5.986418e-18,-1.927135e-17,4.879341e-18,-5.762748e-16,5.777304e-17,1.142176e-15,-1.544578e-15,6.882618e-15,-6.694948e-15,...,1.670047e-16,4.892068e-14,-1.892356e-14,7.491223e-17,-1.389780e-13,-7.483023e-17,-1.828728e-17,-5.986418e-18,-1.499711,1.983688
std,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,...,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.000003e+00,1.116606,4.397062
min,-6.364742e-01,-2.504704e-01,-3.016176e-01,-3.901331e-01,-2.343694e+00,-3.919639e-01,-6.620972e-01,-2.633525e+00,-3.282410e+00,-2.439818e+00,...,-1.432705e+00,-1.421744e+00,-1.967177e+00,-1.663875e+00,-1.590644e+00,-1.769786e+00,-6.321809e-01,-1.668935e+00,-3.000000,-6.000000
25%,-5.585866e-01,-2.496708e-01,-2.872826e-01,-3.462952e-01,-7.303838e-01,-3.466685e-01,-6.620972e-01,-7.403324e-01,-6.361801e-01,-7.646495e-01,...,-1.119262e+00,-3.950523e-01,-3.720938e-01,-7.717303e-01,-1.916616e-01,-8.795586e-01,-6.321809e-01,-8.760112e-01,-2.000000,-2.000000
50%,-4.022075e-01,-2.474320e-01,-2.554716e-01,-3.069064e-01,2.938945e-03,-3.072592e-01,-6.620972e-01,6.889700e-03,5.556234e-02,-2.251823e-03,...,5.893536e-02,-1.301581e-02,-3.164883e-01,1.204140e-01,-1.916616e-01,7.914777e-02,-6.321809e-01,3.018704e-02,-2.000000,2.000000
75%,-4.537341e-02,-2.189672e-01,-1.699055e-01,-2.077363e-01,6.262633e-01,-2.053500e-01,6.774444e-02,6.904922e-01,7.417703e-01,7.019169e-01,...,7.966014e-01,2.967571e-01,1.000842e+00,7.151768e-01,1.207321e+00,7.639380e-01,1.581826e+00,8.231105e-01,-1.000000,6.000000
max,5.400724e+00,5.985888e+00,1.023092e+01,3.860401e+00,2.972896e+00,3.526119e+00,5.906477e+00,2.741224e+00,3.447941e+00,2.301595e+00,...,1.161755e+00,1.787299e+00,1.755793e+00,1.607321e+00,1.207321e+00,1.722644e+00,1.581826e+00,1.729309e+00,0.000000,10.000000


# Model Training for Payment Delay Prediction